## Построение графика социальной динамики по Instagram-аккаунту

На графике будут отображены ключевые слова из комментариев к выбранным Instagram-публикациям за определённый временной промежуток.

### Подключение к базе данных

В .env файле указывается строка подключения к базе данных PostgreSQL (см. .env.example). Данная строка используется в качестве параметра при инициализации класса для работы с БД (Database).

In [57]:
from database import Database
from dotenv import load_dotenv, find_dotenv
from os import getenv

load_dotenv(find_dotenv())

db = Database(getenv('DATABASE_URL'))

a connection to the database has been established


### Получение списка публикаций Instagram

Список записей, содержащих информацию по публикациям пользователя, можно получить с помощью метода get_posts_by_urls класса Database, указав при этом URL публикаций.

*Замечание: информация о публикациях и пользователе должна быть предварительно собрана с использованием InstagramScraper и загружена в БД (DATABASE_URL).*

In [58]:
posts_urls = (
    'https://www.instagram.com/p/CX097AWNTm2/',
    'https://www.instagram.com/p/CX3N4xoNvUO/',
    'https://www.instagram.com/p/CX3ZqJ_NlZg/'
)
posts = db.get_posts_by_urls(posts_urls)
posts

[PostDatabaseModel(id=77, url='https://www.instagram.com/p/CX097AWNTm2/', owner_id=1, picture='https://scontent-frx5-1.cdninstagram.com/v/t51.2885-15/e35/269795273_3156279241316530_9213064171991002480_n.jpg?_nc_ht=scontent-frx5-1.cdninstagram.com&_nc_cat=110&_nc_ohc=sjboVHuQlgYAX8KDGJo&edm=ALQROFkBAAAA&ccb=7-4&ig_cache_key=MjczNTA4MzIwOTM3OTYyNTM5OA%3D%3D.2-ccb7-4&oh=00_AT8UyVuoMwPkPc3P6dZMB4k0Tk7ZuTK2znrdRB0t2DtauA&oe=621BA6F0&_nc_sid=30a2ef', text='С большим вниманием смотрел пресс-конференцию Президента России Владимира Владимировича Путина. Глава государства затронул много важных тем. \n\n🔹ИНСТРУМЕНТЫ ДЛЯ РОСТА ЭКОНОМИКИ \n\nПрезидент обозначил, какие есть драйверы для экономического роста в условиях пандемии, какие существуют инструменты поддержки на федеральном уровне. Наша задача активно входить во все федеральные программы и улучшать инфраструктуру в регионе. В этом году Ульяновская область получила мощную поддержку от Правительства РФ на развитие индустриальных парков. А это н

### Подключение библиотеки srsparser

Для выделения ключевых слов из комментариев к публикациям пользователя Instagram подключается библиотека [srsparser](https://github.com/Text-Analysis/srsparser), содержащая необходимые методы.

In [59]:
%pip install srsparser --upgrade

You should consider upgrading via the 'C:\Users\f0rge\PycharmProjects\posts-scraper\venv\Scripts\python.exe -m pip install --upgrade pip' command.


### Извлечение содержимого комментариев публикаций

Извлечение содержимого комментариев по полученным публикациям пользователя, длина которых больше 20 символов, убрав из содержания упоминания пользователей Instagram.

In [60]:
posts_comments = []
for post in posts:
    post_comments = db.get_comments_by_posts_ids((post.id,))
    filtered_comments_contents = [Database.remove_links_from_comment(comment)
                                  for comment in post_comments
                                  if len(comment.text) > 15]
    posts_comments.append(filtered_comments_contents)

posts_comments_contents = []
for post_comments in posts_comments:
    posts_comments_contents.append([comment.text for comment in post_comments])

posts_comments_contents[0][:3]  # первые 3 комментария 1 публикации

['Алексей Юрьевич,что насчёт канализации на амурской думаете делать,как будем решать проблему?',
 'Как быть тем многодетным которых вообще не ставят на очередь на землю из за того что один ребёнок на территории папы прописан???Глупые критерии!',
 'Остановите ценыыыыыыыы!!!!!!!! Скоро по меру пойдём!😢']

### Обработка данных на естественном языке

Инициализация класса, ответствнного за обработку текста на естественном языке - *NLProcessor*. Он включает в себя методы извлечения ключевых слов с использованием библиотеки [pullenti](https://github.com/pullenti/PullentiPython), построение TF-IDF модели с использованием библиотеки [gensim](https://github.com/RaRe-Technologies/gensim) и др.

In [61]:
from srsparser import NLProcessor

nlp = NLProcessor()

### Получение TF-IDF весов слов комментариев

С помощью метода get_tf_idf_weights класса NLProcessor можно получить TF-IDF веса слов для содержимого каждого комментария.

In [62]:
posts_comments_weights = []
for post_comments_contents in posts_comments_contents:
    posts_comments_weights.append(nlp.get_tf_idf_weights(post_comments_contents))

posts_comments_weights[0][:3]  # tf-idf веса слов первых 3 комментариев 1 публикации

[[['амурской', 0.388],
  ['канализация', 0.388],
  ['делать', 0.33],
  ['насчёт', 0.33],
  ['решать', 0.33],
  ['алексей', 0.296],
  ['думать', 0.296],
  ['юриевич', 0.296],
  ['проблема', 0.254],
  ['быть', 0.214]],
 [['глупый', 0.357],
  ['критерий', 0.357],
  ['папа', 0.357],
  ['прописать', 0.357],
  ['ставить', 0.357],
  ['территория', 0.273],
  ['очередь', 0.251],
  ['земля', 0.219],
  ['который', 0.219],
  ['ребёнок', 0.219],
  ['многодетный', 0.208],
  ['вообще', 0.197]],
 [['поидти', 0.522],
  ['мера', 0.444],
  ['скоро', 0.444],
  ['ценыыыыыыы', 0.444],
  ['остановить', 0.367]]]

### Получение ключевых слов комментариев

С помощью метода get_keywords_pullenti класса NLProcessor можно получить список ключевых слов (и словосочетаний) для каждого комментария публикации.

In [63]:
posts_comments_keywords = []
for post_comments_contents in posts_comments_contents:
    post_comments_keywords = []
    for i in range(len(post_comments_contents)):
        try:
            keywords = nlp.get_keywords_pullenti(post_comments_contents[i])
            post_comments_keywords.append(keywords)
        except Exception:
            post_comments_keywords.append([])
    posts_comments_keywords.append(post_comments_keywords)

posts_comments_keywords[0][:3]

[['ЮРЬЕВИЧ', 'КАНАЛИЗАЦИЯ', 'ДУМАТЬ', 'ДЕЛАТЬ', 'РЕШАТЬ', 'ПРОБЛЕМА'],
 ['ТЕРРИТОРИЯ ПАПЫ',
  'ГЛУПЫЙ КРИТЕРИЙ',
  'СТАВИТЬ',
  'ОЧЕРЕДЬ',
  'ЗЕМЛЯ',
  'РЕБЕНОК',
  'ТЕРРИТОРИЯ',
  'ПАПА',
  'ПРОПИСАТЬ',
  'ГЛУПЫЙ',
  'КРИТЕРИЙ'],
 ['ОСТАНОВИТЬ', 'МЕРА', 'ПОЙТИ']]

### Связь ключевых слов и TF-IDF весов

Рассматривается ключевое слово (или словосочетание) и ему в соответствие ставится TF-IDF вес (если для такого слова он был посчитан). TF-IDF вес для словосочетаний рассчитывается как сумма TF-IDF весов слов, из которых состоит словосочетание.

Поскольку целью является построение графика социальной динамики, то нас интересует список точек, по которым будет строиться этот график. Точка графика будет представлена кортежом, в котором каждый элемент представляет данные для построения графика: ключевое слово, его TF-IDF вес и дата появления.

In [64]:
from typing import List

# для каждой публикации формируется свой список точек (для каждой публикации будет свой график)
posts_points_info = []
# проход по ключевым словам комментариев каждой публикации
for j in range(len(posts_comments_keywords)):
    # массив точек графика, в котором каждый элемент представляет собой список, состоящий из ключевого слова, его TF-IDF веса и даты появления в комментариях
    post_points_info: List[List] = []
    # проход по ключевым словам всех комментариев (список, в котором каждый элемент представлен списком ключевых слов комментария)
    for i in range(len(posts_comments_keywords[j])):
        # если по комментарию было найдено 3 и более ключевых слов
        if len(posts_comments_keywords[j][i]) >= 3:
            # проход по ключевым словам i-того комментария (список, в котором каждый элемент представлен либо ключевым словом, либо ключевым словосочетанием)
            for keyword in posts_comments_keywords[j][i]:
                # проверка, элемент является словом или словосочетанием
                keyphrase_words = keyword.split()
                if len(keyphrase_words) > 1:  # словосочетание
                    tf_idf_sum = 0.0
                    # проход по словам ключевого словосочетания
                    for word in keyphrase_words:
                        # лемматизируем слово (получаем его нормальную форму)
                        normal_word = nlp.get_normal_form(word)
                        for tf_idf_weight in posts_comments_weights[j][i]:
                            # если для ключевого слова нашёлся его TF-IDF вес
                            if normal_word.lower() == tf_idf_weight[0]:
                                tf_idf_sum += tf_idf_weight[1]
                                break
                    if tf_idf_sum > 0.0:
                        post_points_info.append([keyword, tf_idf_sum, posts_comments[j][i].time])
                else:  # слово
                    # лемматизируем слово (получаем его нормальную форму)
                    normal_word = nlp.get_normal_form(keyword)
                    # проход по TF-IDF весам слов всех комментариев (список, в котором каждый элемент представлен списком пар "слово-вес")
                    for tf_idf_weight in posts_comments_weights[j][i]:
                        # если для ключевого слова нашёлся его TF-IDF вес
                        if normal_word.lower() == tf_idf_weight[0]:
                            post_points_info.append([keyword, tf_idf_weight[1], posts_comments[j][i].time])
                            break
    posts_points_info.append(post_points_info)

posts_points_info[0][:3]

[['ЮРЬЕВИЧ',
  0.296,
  datetime.datetime(2021, 12, 23, 10, 30, 24, tzinfo=datetime.timezone.utc)],
 ['КАНАЛИЗАЦИЯ',
  0.388,
  datetime.datetime(2021, 12, 23, 10, 30, 24, tzinfo=datetime.timezone.utc)],
 ['ДУМАТЬ',
  0.296,
  datetime.datetime(2021, 12, 23, 10, 30, 24, tzinfo=datetime.timezone.utc)]]

### Установка библиотеки визуализации данных

В качестве библиотеки для построения графиков выбрана [plotly](https://github.com/plotly/plotly.py). Данная библиотека также требует установки пакета pandas.

In [65]:
%pip install plotly --upgrade
%pip install pandas --upgrade

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\f0rge\PycharmProjects\posts-scraper\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\f0rge\PycharmProjects\posts-scraper\venv\Scripts\python.exe -m pip install --upgrade pip' command.


### Визуализация данных

Преобразуем информацию о ключевых словах в Dataframe pandas для того, чтобы с ним можно было работать в plotly.

In [66]:
import pandas as pd

MIN_RATIO = 0.5

posts_dfs: List[pd.DataFrame] = []
for post_points_info in posts_points_info:
    posts_dfs.append(pd.DataFrame([plot_point for plot_point in post_points_info if plot_point[1] > MIN_RATIO],
                  columns=['Keyword', 'TF-IDF ratio', 'Appearance time']))
posts_dfs[0].head(3)

,Keyword,TF-IDF ratio,Appearance time
0,ТЕРРИТОРИЯ ПАПЫ,0.630,2021-12-23 10:39:19+00:00
1,ГЛУПЫЙ КРИТЕРИЙ,0.714,2021-12-23 10:39:19+00:00
2,ДОБРЫЙ ВЕЧЕР,0.547,2021-12-23 10:54:02+00:00


На основе полученного датафрейма создаём графики, соответствующие выбранным публикациям:

- По оси Oy будут представлены коэффициенты "важности" ключевых слов комментариев.
- По оси Ox будут представлены временные метки, которые относятся к появлению ключевого слова.
- Сама точка на графике будет представлять ключевое слово.

In [67]:
import plotly.express as px

for i in range(len(posts_dfs)):
    fig = px.scatter(posts_dfs[i], x='Appearance time', y='TF-IDF ratio', text='Keyword', title=posts_urls[i])
    fig.show()